In [1]:
import torch
from torch import nn
from torch.nn import functional as F

### 1.自定义块

自定义块通常需要包含如下几个部分：

- **输入：** 输入数据作为其前向传播函数的参数
- **前向传播函数：** 输入通过前向传播函数的作用，产生输出
- **梯度：** 能够计算输出关于输入的梯度
- **模型参数：** 能够存储和访问前向传播计算所需的参数
- **初始化：** 能够对模型参数进行初始化

在构建自定义块时，一般会继承nn.Module类，然后修改__init__()和forward()函数

- \_\_init\_\_()函数中主要实现一些层和参数的初始化
- forward()函数中定义前向传播的实现过程

In [2]:
class MLP(nn.Module):
    def __init__(self) -> None:
        # 调用父类（nn.Module）的构造函数来执行必要的初始化
        super().__init__()
        self.hidden = nn.Linear(20, 256)    # 隐藏层
        self.out = nn.Linear(256, 10)       # 输出层
    
    # 定义模型的前向传播函数
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[-0.0233,  0.0518, -0.2289,  0.1557, -0.0906, -0.1514,  0.0753,  0.0396,
          0.0333, -0.0343],
        [-0.0045, -0.0173, -0.3109,  0.1658, -0.0614, -0.2168,  0.1630,  0.1892,
          0.1727, -0.1910]], grad_fn=<AddmmBackward0>)

### 2.顺序块的实现

In [8]:
class MySequential(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        for idx, module in enumerate(args):
            # args为一个元组，按顺序存储了输入参数
            # _modules的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)
print(net)

MySequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


### 3.前向传播函数

我们可以在前向传播函数中自定义执行流程，引入Python的控制流，不参与梯度计算的固定参数等

In [4]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # 固定权重参数，在训练过程中保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0606, grad_fn=<SumBackward0>)

### 4.自由组合各种块

In [5]:
class NestMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0453, grad_fn=<SumBackward0>)

### 5.练习

##### 5.1 将MySequential中块的存储方式改为Python列表

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        self.list_modules = list(args)

    def forward(self, X):
        for block in self.list_modules:
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)
print(net)

MySequential()


将MySequential中块的存储方式改为list后，正常的前向传播计算还是可以的，但是在访问网络结构和参数方面不如OrderedDict方便

##### 5.2 实现一个块，它以两个块为参数，并返回前向传播中两个网络的串联输出

In [10]:
class MySeriesBlock(nn.Module):
    def __init__(self, block1, block2) -> None:
        super().__init__()
        self.subBlock1 = block1
        self.subBlock2 = block2
    
    def forward(self, X):
        return self.subBlock2(self.subBlock1(X))

block1 = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 128))
block2 = MySequential(nn.Linear(128, 64), nn.ReLU(), nn.Linear(64, 10))
net = MySeriesBlock(block1, block2)
net(X)

tensor([[ 0.1389, -0.0321, -0.1155,  0.0385, -0.0690,  0.0640, -0.0581,  0.0305,
          0.0701, -0.1036],
        [ 0.1367, -0.0241, -0.1106,  0.0356, -0.0747,  0.0659, -0.0517,  0.0326,
          0.0723, -0.1219]], grad_fn=<AddmmBackward0>)

##### 5.3 实现一个网络的多个实例，并以此为基础构建更大的网络

In [12]:
class myBlock(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.block1 = nn.Linear(20, 256)
        self.block2 = nn.ReLU()
        self.block3 = nn.Linear(256, 20)

    def forward(self, X):
        return self.block3(self.block2(self.block1(X)))

class multiBlockNet(nn.Module):
    def __init__(self, subBlock, num) -> None:
        super().__init__()
        self.subBlock = subBlock
        self.num = num
    
    def forward(self, X):
        for i in range(self.num):
            X = self.subBlock(X)
        return X

block = myBlock()
net = multiBlockNet(block, 100)
net(X)

tensor([[-0.0005, -0.0374,  0.0121, -0.0074,  0.0442,  0.0379, -0.1096,  0.0193,
          0.1049, -0.1296,  0.0145, -0.1227,  0.0659, -0.0132,  0.0169,  0.0762,
          0.0078,  0.0133, -0.0348, -0.1132],
        [-0.0005, -0.0374,  0.0121, -0.0074,  0.0442,  0.0379, -0.1096,  0.0193,
          0.1049, -0.1296,  0.0145, -0.1227,  0.0659, -0.0132,  0.0169,  0.0762,
          0.0078,  0.0133, -0.0348, -0.1132]], grad_fn=<AddmmBackward0>)